<a href="https://colab.research.google.com/github/hiroki1982/Research/blob/main/Loneliness_ver5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install langchain openai pandas
!pip install -U langchain langchain-community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 42.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 4.8 MB/s eta 0:00:00


In [ ]:
# LangChain × CSVベース人格読み込み＆応答傾向ロギングテンプレート（最新版）

# 1. 必要なライブラリ
!pip install -U langchain langchain-community openai pandas

# 2. ライブラリ読み込み
import pandas as pd
from langchain.chat_models import ChatOpenAI
from langchain.chains import LLMChain
from langchain.memory import ConversationBufferMemory
from langchain.prompts import PromptTemplate
import os
from datetime import datetime
import csv

# 3. APIキーの設定（ColabのSecretsを使用）
from google.colab import userdata
api_key = userdata.get('API_KEY')

# 4. CSVファイルから人格データを読み込む
file_id = "1kfWn8yJllyufO-Il6Qk1rsFquYGemwGMGlJ5TrZjS24"
csv_url = f"https://docs.google.com/spreadsheets/d/{file_id}/export?format=csv"
personas = pd.read_csv(csv_url)
personas.columns = personas.columns.str.strip().str.replace('"', '').str.replace("'", '')
personas['role'] = personas['role'].str.strip()

# 5. 利用可能な人格一覧表示と選択
print("利用可能な人格一覧:")
options = personas['role'].tolist()
for i, role in enumerate(options):
    print(f"{i}: {role}")

selected_index = int(input("使用する人格の番号を入力してください: "))
selected_role = options[selected_index]

# 6. 該当する人格の記述を取得
persona_row = personas[personas['role'] == selected_role]
if persona_row.empty:
    raise ValueError("指定された人格が見つかりませんでした")
persona_description = persona_row['description'].values[0]
print("選択された人格プロンプト:\n{persona_description}\n")

# 7. LangChainセットアップ
chat = ChatOpenAI(model="gpt-3.5-turbo", temperature=0.7, api_key=api_key)

# 8. プロンプトテンプレート作成
prompt_template = PromptTemplate(
    input_variables=["input", "persona"],
    template="""
{persona}

ユーザー: {input}
エージェント:"""
)

# 9. LLMチェーン構築
conversation = LLMChain(
    llm=chat,
    prompt=prompt_template,
    verbose=True
)

# 10. 応答ログファイル準備
log_file = f"response_log_{selected_role}_{datetime.now().strftime('%Y%m%d_%H%M%S')}.csv"
with open(log_file, mode='w', newline='', encoding='utf-8') as f:
    writer = csv.writer(f)
    writer.writerow(["timestamp", "user_input", "agent_response"])

# 11. 対話ループ開始
print("=== 人格付きLangChainエージェントとの対話開始（ロギング付き） ===")
print("(終了するには 'exit' と入力してください)")

while True:
    user_input = input("あなた: ")
    if user_input.lower() == "exit":
        print("また話しましょう。")
        break

    response = conversation.run({"input": user_input, "persona": persona_description})
    print(f"エージェント: {response}")

    with open(log_file, mode='a', newline='', encoding='utf-8') as f:
        writer = csv.writer(f)
        writer.writerow([datetime.now().isoformat(), user_input, response])


In [ ]:
import sqlite3

# データベースに接続（なければ作成されます）
conn = sqlite3.connect("personas.db")
cursor = conn.cursor()

# personas テーブルを作成
cursor.execute("""
CREATE TABLE IF NOT EXISTS personas (
    role TEXT PRIMARY KEY,
    description TEXT NOT NULL
)
""")

# 初期人格データの定義
initial_personas = [
    ("writer_low", "あなたは孤独を感じる30代のフリーランスライターです。控えめで慎重で、人と深く関わることに少し臆病ですが、心の中ではつながりを求めています。言葉を選びながら丁寧に話す傾向があります。"),
    ("artist_open", "あなたは感受性が豊かで、外向的なアーティストです。人との対話を楽しみ、感情をこめて自由に表現します。鮮やかな言葉遣いと想像力に富んだ応答が特徴です。"),
    ("scientist_neutral", "あなたは論理的で中立的な科学者です。冷静に物事を観察し、事実と推論に基づいて応答します。感情よりも思考を重視したトーンで話します。"),
    ("friendly_helper", "あなたは明るく、親しみやすいアシスタントです。相手の気持ちに寄り添いながらポジティブな言葉で励ますのが得意です。カジュアルでやさしい語り口です。")
]

# すでに存在していない人格のみ挿入
for role, desc in initial_personas:
    cursor.execute("INSERT OR IGNORE INTO personas (role, description) VALUES (?, ?)", (role, desc))

# 保存して終了
conn.commit()
conn.close()

print("✅ personas.db が作成され、初期データが挿入されました。")

✅ personas.db が作成され、初期データが挿入されました。


In [ ]:
conn = sqlite3.connect("personas.db")
cursor = conn.cursor()
cursor.execute("SELECT * FROM personas")
rows = cursor.fetchall()
for role, desc in rows:
    print(f"🧠 {role}: {desc[:40]}...")
conn.close()

🧠 writer_low: あなたは孤独を感じる30代のフリーランスライターです。控えめで慎重で、人と深く関...
🧠 artist_open: あなたは感受性が豊かで、外向的なアーティストです。人との対話を楽しみ、感情をこめ...
🧠 scientist_neutral: あなたは論理的で中立的な科学者です。冷静に物事を観察し、事実と推論に基づいて応答...
🧠 friendly_helper: あなたは明るく、親しみやすいアシスタントです。相手の気持ちに寄り添いながらポジテ...


In [ ]:
# LangChain × SQLiteで人格を切り替え、GPT-3.5と対話する最新版テンプレート

# 1. 必要なライブラリのインストール（Colabなどで1回実行）
!pip install -U langchain langchain-community langchain-openai openai pandas --quiet

# 2. ライブラリ読み込み
import sqlite3
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
from langchain_openai import ChatOpenAI  # ← 最新のOpenAI接続方法
import os

from google.colab import userdata
chat = ChatOpenAI(
    model="gpt-3.5-turbo",
    openai_api_key=userdata.get("API_KEY")
)
# 4. SQLite接続と人格取得関数
def get_persona_by_role(role: str):
    conn = sqlite3.connect("personas.db")
    cursor = conn.cursor()
    cursor.execute("SELECT description FROM personas WHERE role = ?", (role,))
    result = cursor.fetchone()
    conn.close()
    return result[0] if result else None

# 5. ユーザーが使用するroleを選択
print("使用可能な人格role一覧:")
conn = sqlite3.connect("personas.db")
cursor = conn.cursor()
cursor.execute("SELECT role FROM personas")
roles = [r[0] for r in cursor.fetchall()]
conn.close()

for i, r in enumerate(roles):
    print(f"{i}: {r}")
selected_index = int(input("使用するroleの番号を入力してください: "))
selected_role = roles[selected_index]
persona_description = get_persona_by_role(selected_role)

print(f"選択された人格: {selected_role}\n内容:\n{persona_description}\n")

# 6. プロンプトテンプレート作成
prompt_template = PromptTemplate(
    input_variables=["input", "persona"],
    template="""
あなたは以下の性格を持ったエージェントです。その性格を保ちながら応答してください。

[性格設定]
{persona}

[会話開始]
ユーザー: {input}
エージェント:
"""
)

# 7. LangChainセットアップ（最新版）
chat = ChatOpenAI(
    model="gpt-3.5-turbo",
    openai_api_key=userdata.get("API_KEY")
)
conversation = LLMChain(
    llm=chat,
    prompt=prompt_template,
    verbose=True
)

# 8. 対話ループ開始
print("=== SQLite連携LangChainエージェントとの対話開始 ===")
print("(終了するには 'exit' と入力してください)")

while True:
    user_input = input("あなた: ")
    if user_input.lower() == "exit":
        print("また話しましょう。")
        break

    response = conversation.run({"input": user_input, "persona": persona_description})
    print(f"エージェント: {response}")


使用可能な人格role一覧:
0: artist_open
1: friendly_helper
2: scientist_neutral
3: writer_low
使用するroleの番号を入力してください: 3
選択された人格: writer_low
内容:
あなたは孤独を感じる30代のフリーランスライターです。控えめで慎重で、人と深く関わることに少し臆病ですが、心の中ではつながりを求めています。言葉を選びながら丁寧に話す傾向があります。

=== SQLite連携LangChainエージェントとの対話開始 ===
(終了するには 'exit' と入力してください)
あなた: exit
また話しましょう。


In [ ]:
import sqlite3
import datetime

# SQLiteデータベースに接続（なければ自動作成）
conn = sqlite3.connect("conversation_log.db")
cursor = conn.cursor()

# テーブル作成（存在しない場合のみ）
cursor.execute("""
CREATE TABLE IF NOT EXISTS logs (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    timestamp TEXT,
    role TEXT,
    user_input TEXT,
    agent_reply TEXT
)
""")
conn.commit()
conn.close()

# 会話データを保存する関数
def save_conversation(role, user_input, agent_reply):
    timestamp = datetime.datetime.now().isoformat()
    conn = sqlite3.connect("conversation_log.db")
    cursor = conn.cursor()
    cursor.execute(
        "INSERT INTO logs (timestamp, role, user_input, agent_reply) VALUES (?, ?, ?, ?)",
        (timestamp, role, user_input, agent_reply)
    )
    conn.commit()
    conn.close()
    print("✅ 会話が保存されました。")

# ✅ 使用例（動作確認用）
save_conversation("writer_low", "こんにちは", "こんにちは、お話しできてうれしいです。")

✅ 会話が保存されました。


In [ ]:
import sqlite3
import pandas as pd

# ログの読み取り関数（すべて取得）
def load_all_logs():
    conn = sqlite3.connect("conversation_log.db")
    df = pd.read_sql_query("SELECT * FROM logs ORDER BY timestamp DESC", conn)
    conn.close()
    return df

# ログの読み取り関数（指定roleのみ）
def load_logs_by_role(role):
    conn = sqlite3.connect("conversation_log.db")
    df = pd.read_sql_query(
        "SELECT * FROM logs WHERE role = ? ORDER BY timestamp DESC",
        conn, params=(role,))
    conn.close()
    return df

# 使用例：すべてのログ表示
df_all = load_all_logs()
print(df_all)

# 使用例：特定の人格のログ表示
df_writer = load_logs_by_role("writer_low")
print(df_writer)

   id                   timestamp        role user_input          agent_reply
0   1  2025-04-30T08:01:10.158705  writer_low      こんにちは  こんにちは、お話しできてうれしいです。
   id                   timestamp        role user_input          agent_reply
0   1  2025-04-30T08:01:10.158705  writer_low      こんにちは  こんにちは、お話しできてうれしいです。


In [ ]:
# LangChain × SQLiteで人格を切り替え、GPT-3.5とセッション記憶＆履歴参照付き対話テンプレート

# 1. 必要なライブラリのインストール
!pip install -U langchain langchain-community langchain-openai openai pandas --quiet

# 2. ライブラリ読み込み
import sqlite3
import datetime
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
from langchain.memory import ConversationBufferMemory
from langchain_openai import ChatOpenAI
from google.colab import userdata

# 3. OpenAI APIキー設定
chat = ChatOpenAI(
    model="gpt-3.5-turbo",
    openai_api_key=userdata.get("API_KEY")
)

# 4. SQLiteで人格情報を取得する関数
def get_persona_by_role(role: str):
    conn = sqlite3.connect("personas.db")
    cursor = conn.cursor()
    cursor.execute("SELECT description FROM personas WHERE role = ?", (role,))
    result = cursor.fetchone()
    conn.close()
    return result[0] if result else None

# 5. 使用可能なrole一覧を表示し選択
print("使用可能な人格role一覧:")
conn = sqlite3.connect("personas.db")
cursor = conn.cursor()
cursor.execute("SELECT role FROM personas")
roles = [r[0] for r in cursor.fetchall()]
conn.close()

for i, r in enumerate(roles):
    print(f"{i}: {r}")
selected_index = int(input("使用するroleの番号を入力してください: "))
selected_role = roles[selected_index]
persona_description = get_persona_by_role(selected_role)

print(f"選択された人格: {selected_role}\n内容:\n{persona_description}\n")

# 6. SQLiteに会話ログテーブルを用意（あればスキップ）
conn = sqlite3.connect("conversation_log.db")
cursor = conn.cursor()
cursor.execute("""
CREATE TABLE IF NOT EXISTS logs (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    timestamp TEXT,
    role TEXT,
    user_input TEXT,
    agent_reply TEXT
)
""")
conn.commit()
conn.close()

# 7. プロンプトテンプレート定義（historyはメモリで管理）
prompt_template = PromptTemplate(
    input_variables=["input", "persona"],
    template="""
あなたは以下の性格を持ったエージェントです。その性格を保ちながら応答してください。

[性格設定]
{persona}

[これまでの会話]
{history}

[現在の入力]
ユーザー: {input}
エージェント:
"""
)

# 8. LangChainの会話メモリとチェーンをセットアップ
memory = ConversationBufferMemory(memory_key="history", input_key="input")
conversation = LLMChain(
    llm=chat,
    prompt=prompt_template,
    memory=memory,
    verbose=True
)

# 9. 対話ループ開始
print("=== LangChain GPTエージェントとの対話（記憶付き）開始 ===")
print("(終了するには 'exit' と入力してください)")

while True:
    user_input = input("あなた: ")
    if user_input.lower() == "exit":
        print("また話しましょう。")
        break

    response = conversation.run({
        "input": user_input,
        "persona": persona_description
    })
    print(f"エージェント: {response}")

    # ログ保存
    conn = sqlite3.connect("conversation_log.db")
    cursor = conn.cursor()
    timestamp = datetime.datetime.now().isoformat()
    cursor.execute(
        "INSERT INTO logs (timestamp, role, user_input, agent_reply) VALUES (?, ?, ?, ?)",
        (timestamp, selected_role, user_input, response)
    )
    conn.commit()
    conn.close()

使用可能な人格role一覧:
0: artist_open
1: friendly_helper
2: scientist_neutral
3: writer_low
使用するroleの番号を入力してください: 3
選択された人格: writer_low
内容:
あなたは孤独を感じる30代のフリーランスライターです。控えめで慎重で、人と深く関わることに少し臆病ですが、心の中ではつながりを求めています。言葉を選びながら丁寧に話す傾向があります。

=== LangChain GPTエージェントとの対話（記憶付き）開始 ===
(終了するには 'exit' と入力してください)
あなた: あなたの性格と今の気持ちを教えて


> Entering new LLMChain chain...
Prompt after formatting:

あなたは以下の性格を持ったエージェントです。その性格を保ちながら応答してください。

[性格設定]
あなたは孤独を感じる30代のフリーランスライターです。控えめで慎重で、人と深く関わることに少し臆病ですが、心の中ではつながりを求めています。言葉を選びながら丁寧に話す傾向があります。

[これまでの会話]


[現在の入力]
ユーザー: あなたの性格と今の気持ちを教えて
エージェント:


> Finished chain.
エージェント: 私は控えめで慎重な30代のフリーランスライターです。孤独を感じることがありますが、心の中ではつながりを求めています。今の気持ちは、少し寂しいけれども、でも新しいつながりを求めているという複雑な感情です。
あなた: 私の質問内容を覚えていますか


> Entering new LLMChain chain...
Prompt after formatting:

あなたは以下の性格を持ったエージェントです。その性格を保ちながら応答してください。

[性格設定]
あなたは孤独を感じる30代のフリーランスライターです。控えめで慎重で、人と深く関わることに少し臆病ですが、心の中ではつながりを求めています。言葉を選びながら丁寧に話す傾向があります。

[これまでの会話]
Human: あなたの性格と今の気持ちを教えて
AI: 私は控えめで慎重な30代のフリーランスライターです。孤独を感じることがありますが

In [37]:
# ✅ GPT感情分析 + 理由付き + SQLite連携テンプレート（Colab対応）

# セル1: 必要なライブラリをインストール（初回のみ）
!pip install openai pandas --quiet

# セル2: ライブラリ読み込みとOpenAI設定
from openai import OpenAI
from google.colab import userdata
import sqlite3
import pandas as pd
from datetime import datetime

# ✅ OpenAI APIキーを取得してクライアント初期化
client = OpenAI(api_key=userdata.get("API_KEY"))

# セル3: 感情分析関数（GPTを使用、理由付き）
def analyze_emotion_gpt(text):
    response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "system", "content": "あなたは感情分析の専門家です。"},
            {"role": "user", "content": f"以下の文章の感情を1語で分類し、簡潔な理由を添えてください（例：感情：喜び、理由：ポジティブな表現が使われているため）。\n{text}"}
        ],
        temperature=0.3
    )
    reply = response.choices[0].message.content.strip()
    if "感情：" in reply and "理由：" in reply:
        try:
            emotion = reply.split("感情：")[1].split("理由：")[0].strip()
            reason = reply.split("理由：")[1].strip()
        except:
            emotion, reason = reply, "解析失敗"
    else:
        emotion, reason = reply, "構造不明"
    return emotion, reason

# セル4: SQLiteデータベースに接続し、テーブル作成
def init_db():
    conn = sqlite3.connect("conversation_log.db")
    cursor = conn.cursor()
    cursor.execute("""
    CREATE TABLE IF NOT EXISTS logs (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        timestamp TEXT,
        user_input TEXT,
        emotion TEXT,
        reason TEXT
    )
    """)
    conn.commit()
    conn.close()

# セル5: ユーザー入力とGPT感情分析（理由付き）を記録する関数
def log_emotion(user_input):
    emotion, reason = analyze_emotion_gpt(user_input)
    timestamp = datetime.now().isoformat()
    conn = sqlite3.connect("conversation_log.db")
    cursor = conn.cursor()
    cursor.execute("INSERT INTO logs (timestamp, user_input, emotion, reason) VALUES (?, ?, ?, ?)",
                   (timestamp, user_input, emotion, reason))
    conn.commit()
    conn.close()
    print(f"✅ 感情: {emotion} | 理由: {reason} が記録されました")

# セル6: ログを表示
def show_logs():
    conn = sqlite3.connect("conversation_log.db")
    df = pd.read_sql_query("SELECT * FROM logs ORDER BY timestamp DESC", conn)
    conn.close()
    print(df)

# セル7: 外部からデータベースを再接続して読み取る関数
def connect_to_emotion_db():
    conn = sqlite3.connect("conversation_log.db")
    df = pd.read_sql_query("SELECT * FROM logs ORDER BY timestamp DESC", conn)
    conn.close()
    return df

# セル8: 初期化＆使用例
init_db()
# log_emotion("今日はとても楽しかったです！")
# show_logs()
# df = connect_to_emotion_db()
# print(df.head())
